# Atividade de Laboratório 1
### Integrantes
- Leonardo Mallmann
- Caetano Muller
- Euzébio Hensel

In [79]:
import urllib.request
from bs4 import BeautifulSoup
from urllib.error import URLError, HTTPError, ContentTooShortError
import re
import csv

In [103]:
def download(url, user_agent='wswp', charset='utf-8'):
  print('Downloading:', url)
  request = urllib.request.Request(url)
  request.add_header('User-agent', user_agent)
  try:
    resp = urllib.request.urlopen(request)
    cs = resp.headers.get_content_charset()
    if not cs:
        cs = charset
    html = resp.read().decode(cs)
  except (URLError, HTTPError, ContentTooShortError) as e:
    print('Download error:', e.reason)
  return html

In [94]:
def scrap(html_content):
  soup = BeautifulSoup(html_content, 'html5lib')
  rows = soup.select('table tbody tr')
  field_names = []
  field_values = []
  for row in rows:
    c = row.select('.w2p_fl label');
    field_names.append(c[0].string.strip().replace(':', ''));
    v = row.select('.w2p_fw');
    if not v[0].string:
      if len(re.findall('<img', str(v[0]))) != 0:
        img_src = v[0].select('img')[0];
        field_values.append(img_src['src'])
        continue
      if len(re.findall('<div', str(v[0]))) != 0:
        a_links = v[0].select('div a');
        neighbours = [a_tag.string.strip() for a_tag in a_links]
        field_values.append(neighbours);
        continue
      field_values.append('None')
      continue
    field_values.append(v[0].string)
    
  return field_values, field_names
    

### Exemplo de inserção no CSV

In [112]:
def insert_headers(fieldnames):
  with open('places.csv', 'w', newline='\n') as places_csv:
    writer = csv.DictWriter(places_csv, fieldnames=fieldnames)
    writer.writeheader()

def insert_data(data_row, fieldnames):
  with open('places.csv', 'a', newline='\n') as places_csv:
    writer = csv.DictWriter(places_csv, fieldnames=fieldnames)
    
    writer.writerow({
      f_name: f_value
      for f_name, f_value in zip(fieldnames,data_row)
    }) # or dict(zip(keys, values))

In [ ]:
with open('names.csv', 'r', newline='') as csvfile:
  reader = csv.DictReader(csvfile)
  for row in reader:
    print(row)
    if str(['beans', 'marsh']) == row['first_name']:
      print('match')

In [110]:
def crawl_sitemap(url):
  sitemap = download(url)
  links = re.findall('\d(\/.*)<\/loc>', sitemap)
  for index, link in enumerate(links):
    html = download(f'http://localhost:8000{link}')
    f_values, f_names = scrap(html)
    if index==0:
      insert_headers(f_names)
    insert_data(f_values, f_names)
    if index==4:
      break

In [113]:
crawl_sitemap('http://localhost:8000/places/static/sitemap.xml');

Downloading: http://localhost:8000/places/static/sitemap.xml
Downloading: http://localhost:8000/places/default/view/Afghanistan-1


TypeError: writeheader() takes 1 positional argument but 2 were given